In [ ]:
import pandas as pd

orders = pd.read_csv("/content/orders.csv")


In [ ]:
orders.head()
orders.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [ ]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
users = pd.read_json("/content/users.json")


In [ ]:
users.head()
users.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [ ]:
import sqlite3
import pandas as pd


In [ ]:
conn = sqlite3.connect("/content/restaurants.sql")
cursor = conn.cursor()


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("/content/restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()


In [ ]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
orders.head()
users.head()
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
orders.shape, users.shape, restaurants.shape


((10000, 6), (3000, 4), (500, 4))

In [ ]:
order_user = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [ ]:
order_user.head()
order_user.shape


(10000, 9)

In [ ]:
final_df = pd.merge(
    order_user,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [ ]:
final_df.head()
final_df.shape


(10000, 12)

In [ ]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])


/tmp/ipython-input-486593379.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


In [ ]:
final_df.isnull().sum()


,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
restaurant_name_x,0
name,0
city,0
membership,0
restaurant_name_y,0


In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


MCQ

In [ ]:
#q1 Which city has the highest total revenue (total_amount) from Gold members?
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [ ]:
#q2Which cuisine has the highest average order value across all orders?
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [ ]:
#q3How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_spend = final_df.groupby("user_id")["total_amount"].sum()
(user_spend > 1000).sum()


np.int64(2544)

In [ ]:
#q4Which restaurant rating range generated the highest total revenue?
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0,3.5,4.0,4.5,5.0]
)

final_df.groupby("rating_range")["total_amount"].sum()


/tmp/ipython-input-253509727.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum()


,total_amount
rating_range,
"(3.0, 3.5]",1881754.57
"(3.5, 4.0]",1717494.41
"(4.0, 4.5]",1960326.26
"(4.5, 5.0]",2197030.75


In [ ]:
#q5Among Gold members, which city has the highest average order value?
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [ ]:
#q6Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
)


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
#q7What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
gold_orders = final_df[final_df["membership"]=="Gold"].shape[0]
total_orders = final_df.shape[0]

round((gold_orders/total_orders)*100)


50

In [ ]:
#q8Which restaurant has the highest average order value but less than 20 total orders?
final_df.groupby("restaurant_name_y").agg(
    avg_order_value=("total_amount","mean"),
    total_orders=("order_id","count")
).query("total_orders < 20").sort_values("avg_order_value", ascending=False)


,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [ ]:
#q9Which combination contributes the highest revenue?
final_df.groupby(
    ["membership","cuisine"]
)["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
#q10During which quarter of the year is the total revenue highest?
final_df["quarter"] = final_df["order_date"].dt.quarter

final_df.groupby("quarter")["total_amount"].sum()


,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66
